In [101]:
import pandas as pd
import numpy as np

title_akas = pd.read_csv('../downloads/title.akas.tsv', delimiter='\t', dtype={'isOriginalTitle': object})
title_basics = pd.read_csv('../downloads/title.basics.tsv', delimiter='\t', dtype={'isAdult': object})
title_crew = pd.read_csv('../downloads/title.crew.tsv', delimiter='\t', dtype={'directors': 'str'})
title_ratings = pd.read_csv('../downloads/title.ratings.tsv', delimiter='\t')
name_basics = pd.read_csv('../downloads/name.basics.tsv', delimiter='\t')

In [103]:
# filter by titleType

title_basics = title_basics[(title_basics['titleType'] == 'movie') | (title_basics['titleType'] == 'tvMovie')]
title_basics.reset_index(inplace=True)

# filter by isOriginalTitle

title_akas = title_akas[(title_akas['isOriginalTitle'] == '1') | (title_akas['isOriginalTitle'] == '\\N')]
title_akas.reset_index(inplace=True)


In [104]:
# remove duplicate values with null value in 'isOriginalTitle' column

duplicates = title_akas[title_akas.duplicated(['titleId'])]
duplicates = duplicates[(duplicates['isOriginalTitle'] == '\\N') | (duplicates['attributes'] != '\\N')]

title_akas.drop(labels=duplicates.index, axis=0, inplace=True)
title_akas.reset_index(inplace=True)

In [106]:
duplicates = title_akas[title_akas.duplicated(['titleId'])]

title_akas.drop(labels=duplicates.index, axis=0, inplace=True)
title_akas.reset_index(inplace=True, drop=True)

In [107]:
# drop useless columns 

title_akas.drop(labels=['ordering', 'types', 'attributes', 'title', 'isOriginalTitle'], axis=1, inplace=True)
title_basics.drop(labels=['endYear', 'titleType'], axis=1, inplace=True)
title_crew.drop(labels=['writers'], axis=1, inplace=True)
title_ratings.drop(labels=['numVotes'], axis=1, inplace=True)
name_basics.drop(labels=['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], axis=1, inplace=True)

In [108]:
title_akas['language'].value_counts()

\N     1790606
en           4
tr           1
yue          1
da           1
ca           1
Name: language, dtype: int64

:warning:  language column values have little to none importance, hence not using this dataframe

In [ ]:
# If it is to remove movies before a certain year
# Remove from title_basics


#REVIEW
# I don't think we should remove them 
# Bia

In [118]:
df = title_basics.merge(
  title_crew, on='tconst', how='left').merge(
    title_ratings, on='tconst', how='left')

In [119]:
# explode movie rows with more than one director into multiple rows, each one with only one director code

df = df.assign(directors=df.directors.str.split(',')).explode('directors')

# merge dataframes to get directors names

df = pd.merge(
    df, 
    name_basics, 
    left_on='directors', 
    right_on='nconst',
    how='left')

# only the 'primaryName' column is relevant

df.drop(labels=['directors', 'nconst'], axis=1, inplace=True)

df

,index,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,primaryName
0,8,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,Romance,5.2,Alexander Black
1,498,tt0000502,Bohemios,Bohemios,0,1905,100,\N,3.7,Ricardo de Baños
2,570,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,Charles Tait
3,587,tt0000591,The Prodigal Son,L'enfant prodigue,0,1907,90,Drama,4.0,Michel Carré
4,610,tt0000615,Robbery Under Arms,Robbery Under Arms,0,1907,\N,Drama,4.1,Charles MacMahon
...,...,...,...,...,...,...,...,...,...,...
841135,9262947,tt9916692,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015,66,Drama,NaN,Andrzej Bartnikowski
841136,9262954,tt9916706,Dankyavar Danka,Dankyavar Danka,0,2013,\N,Comedy,NaN,Kanchan Nayak
841137,9262965,tt9916730,6 Gunn,6 Gunn,0,2017,116,\N,8.4,Kiran Gawade
841138,9262975,tt9916754,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,0,2013,49,Documentary,NaN,Vinicius Augusto Bozzo


In [124]:
x = df.groupby(['tconst'])['primaryName'].agg({'primaryName': ', '.join})

x

SpecificationError: nested renamer is not supported

In [121]:
x['primaryName'].head()

<class 'pandas.core.frame.DataFrame'>
Index: 256976 entries, ''Knife'' Sotelo to Þórunn Lárusdóttir
Data columns (total 1 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   primaryName  256976 non-null  object
dtypes: object(1)
memory usage: 3.9+ MB


In [113]:

x = df.pivot_table(index='tconst', values='primaryName', aggfunc=lambda x: ', '.join(x))

x
#x.head()

,primaryName
tconst,
tt0000009,"tconst, primaryName"
tt0000502,"tconst, primaryName"
tt0000574,"tconst, primaryName"
tt0000591,"tconst, primaryName"
tt0000615,"tconst, primaryName"
...,...
tt9916680,"tconst, primaryName"
tt9916692,"tconst, primaryName"
tt9916706,"tconst, primaryName"


# TODO:
- remover filmes mais antigos que x data
- filtrar filmes apenas em en 
  - not enough values
- remover filmes sem sinopse (isto é quando formos fazer o webscrapping ig)
